## Grab SafeGraph data from AWS 

#### code from ryan@safegraph.com

In [1]:
from bs4 import BeautifulSoup as bs
from bs4 import Tag, NavigableString
import re
import requests as r
import pandas as pd
import numpy as np
import datetime
import csv
import os
import sys
from state_cleaner import *
from selenium import webdriver
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [10]:
#Configure the aws profile **check the slack for new access keys weekly!
#! aws configure --profile safegraph

/Users/samismalling/Documents/mobility-report-data-extractor-master/CovidMobile/notebooks


In [ ]:
#Downloads all current available data social distancing data: *takes a while to run
#! aws s3 sync s3://sg-c19-response/social-distancing/v2/ /Users/samismalling/Documents/mobility-report-data-extractor-master/CovidMobile/SafeGraph_data --profile safegraph

In [63]:
df = pd.read_csv("/Users/samismalling/Documents/SafeGraph_data/2020/01/01/2020-01-01-social-distancing.csv.gz")

In [4]:
#Create list of dates to loop through
dates_obj = list(pd.date_range(start="2020-01-01",end="2020-02-01"))
dates_slashes = [x.strftime("%Y/%m/%d") for x in dates_obj]
dates_dashes = [x.strftime("%Y-%m-%d") for x in dates_obj]

In [ ]:
#Read in all of the csvs for each date:
sdm_df = pd.DataFrame()
for date1, date2 in zip(dates_slashes, dates_dashes):
    df = pd.read_csv("/Users/samismalling/Documents/SafeGraph_data/{}/{}-social-distancing.csv.gz".format(date1,date2))
    sdm_df = sdm_df.append(df)
    print(date1)

2020/01/01
2020/01/02
2020/01/03
2020/01/04
2020/01/05
2020/01/06
2020/01/07
2020/01/08
2020/01/09
2020/01/10
2020/01/11


In [64]:
sdm_df = df[["origin_census_block_group", "date_range_start", "date_range_end", 
                                 "device_count", "completely_home_device_count", "part_time_work_behavior_devices", 
                                 "full_time_work_behavior_devices"]]
# convert numerical columns
int_columns = ['device_count', 'completely_home_device_count']
for int_col in int_columns:
    sdm_df[int_col] = sdm_df[int_col].astype('int')

#datetime columns
sdm_df['date_start'] = sdm_df.date_range_start.str.slice(start=0, stop=10)
sdm_df['dt'] = pd.to_datetime(sdm_df['date_start'])
sdm_df['week'] = sdm_df.dt.dt.week 

# edit the census block group to get county fips
sdm_df = sdm_df.astype({'origin_census_block_group':'str'})
sdm_df['county_fips'] = sdm_df.origin_census_block_group.str.slice(start=0, stop=5) # county is the first 5 digits of the CBG

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

,origin_census_block_group,date_range_start,date_range_end,device_count,completely_home_device_count,part_time_work_behavior_devices,full_time_work_behavior_devices,date_start,dt,week,county_fips
0,10330210004,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,93,20,8,2,2020-01-01,2020-01-01,1,10330
1,10730049022,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,157,40,8,5,2020-01-01,2020-01-01,1,10730
2,11210118001,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,96,38,6,2,2020-01-01,2020-01-01,1,11210
3,11250106021,2020-01-01T00:00:00-06:00,2020-01-02T00:00:00-06:00,486,148,50,22,2020-01-01,2020-01-01,1,11250
4,21880002003,2020-01-01T00:00:00-09:00,2020-01-02T00:00:00-09:00,10,4,1,1,2020-01-01,2020-01-01,1,21880


In [65]:
sdm_columns = ['device_count', 'completely_home_device_count']  # 'part_time_work_behavior_devices', 'full_time_work_behavior_devices'
geo_groupby= 'county_fips'
df_by_county = sdm_df.groupby([geo_groupby, 'week', 'date_start'])[sdm_columns].sum().sort_values(by=[geo_groupby, 'week', 'date_start'], ascending=True).reset_index()
df_by_county.head()

,county_fips,week,date_start,device_count,completely_home_device_count
0,10001,1,2020-01-01,11472,3924
1,10003,1,2020-01-01,30270,10734
2,10005,1,2020-01-01,13572,4537
3,10010,1,2020-01-01,5501,1578
4,10030,1,2020-01-01,20761,5997


In [ ]:
# compute new metrics
ca_by_county['leaving_home'] = ca_by_county['device_count'] - ca_by_county['completely_home_device_count']
ca_by_county['pct_leaving_home'] = ca_by_county['leaving_home'] / ca_by_county['device_count'] * 100
  
ca_by_county.head()

In [ ]:
#Create list of dates
dates_obj = list(pd.date_range(start="2020-01-01",end="2020-05-10"))
dates = [x.strftime("%m/%d/%Y") for x in dates_obj]

In [15]:
for date in dates:
    path = "/Users/samismalling/Documents/SafeGraph_data/s3://sg-c19-response/social-distancing/v2/2020/*/*/*.csv.gz")
    df = pd.read_csv("/Users/samismalling/Documents/SafeGraph_data/s3://sg-c19-response/social-distancing/v2/2020/*/*/*.csv.gz")



FileNotFoundError: [Errno 2] No such file or directory: '/Users/samismalling/Documents/SafeGraph_data/s3://sg-c19-response/social-distancing/v2/2020/*/*/*.csv.gz'